# Convolutional Neural Networks

<img src="mnist_sample.png", height=200, width=200>

In [1]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/usr/local/bin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


## Task 1: Load the Data

Load the entire MNIST set from Tensorflow's dataset.

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Set up Parameter

In [3]:
# set batchSz batch gradient descent
batchSz = 50

# set hiddenSz for the hidden layer
hiddenSz = 1000 

# set learning rate 
l = 0.0004

# Placeholders:

In [4]:
# Images and their corresponding labels
img = tf.placeholder(tf.float32, [None, 28, 28, 1])
ans = tf.placeholder(tf.float32, [None, 10])

## Architecture
* First convolution layer with 5x5 filter with 1 in channel and 32 in depth and SAME padding
* ReLU
* Maxpooling layer with 2x2 filter and SAME padding
* Second Convolution layer with 5x5 filter 32 in channel and 64 in depth
* ReLU
* Maxpooling layer with 2x2 filter and SAME padding
* Feed-forward layer from 3136 (you should have 3136 parameters at this point) to 1000
* ReLU
* Feed-forward layer from 1000 to 10
* Softmax/Sigmoid


In [5]:
# First Convolution layer with 5x5 filters, 1 in channel, and 32 depth 
# ReLU
conv1 = tf.layers.conv2d(inputs=img, 
                         filters=32, 
                         kernel_size=[5, 5], 
                         padding="same", 
                         activation=tf.nn.relu)
#print(conv1.shape)

# Maxpooling layer with 2x2 filter and SAME padding
pool1 = tf.layers.max_pooling2d(inputs=conv1, 
                                pool_size=[2, 2], 
                                strides=2, 
                                padding="same")
#print(pool1.shape)

# Second convolution layer with 5x5 filters, 32 in channel, and 64 in depth
# ReLU
conv2 = tf.layers.conv2d(inputs=pool1, 
                         filters=64, 
                         kernel_size=[5, 5], 
                         padding="same", 
                         activation=tf.nn.relu)
#print(conv2.shape)
# Maxpooling layer with 2x2 filter and SAME padding
pool2 = tf.layers.max_pooling2d(inputs=conv2, 
                                pool_size=[2, 2], 
                                strides=2, 
                                padding="same")

#print(pool2.shape)

#Feed-forward layer from 3136 (you should have 3136 parameters at this point) to 1000
pool2_flat = tf.reshape(pool2, [-1, 64*7*7])

# initialize W and bw for the first hidden layer: W should be (64*7*7 by hiddensize), bw should be (hiddensize by 1)
# Feed-forward layer from 1000 to 10
fc1 = tf.layers.dense(pool2_flat, hiddenSz)

# initialize V and bv for the final layer: V should be (hiddensize by 10), and bv should be (10 by 1)
# Softmax/Sigmoid
# Output, class prediction
out = tf.layers.dense(fc1, 10)
prediction = tf.nn.softmax(out)

## Forward and Backward Pass
Write the forward and backward pass in tensorflow
* Use cross entropy loss
* Use AdamOptimizer

In [6]:
# Forward passs -- compute the cross entropy and output probabilities
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=ans))
optimizer = tf.train.AdamOptimizer(learning_rate=l)


# Backward pass
# Note: you don't need to compute the gradient 
# Simply use the tf.train.AdamOptimizer() function
train = optimizer.minimize(loss_op)


# Compute Accuracy
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(ans, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



## Training the Network and Testing our Results
Train on 2000 batches

In [7]:
start = time.time()

step = 100
for i in range(2000):
    if i % step == 0: sstart = time.time()  
    imgs, anss = mnist.train.next_batch(batchSz)
    sess.run(train, feed_dict={img: imgs.reshape([batchSz,28,28,1]), ans: anss})

# Batch size for evaluation performance
batchSz = 100
        
# Accuracy for train data
trainAcc = 0
batchNum =  int(mnist.train.num_examples/batchSz)
for i in range(batchNum):
    imgs, anss= mnist.train.next_batch(batchSz)
    trainAcc += sess.run(accuracy, feed_dict={img: imgs.reshape([batchSz,28,28,1]), ans: anss})
print ("Train Accuracy: %r" % round(trainAcc/batchNum, 3))

# Accuracy for test data
testAcc = 0
batchNum = int(mnist.test.num_examples/batchSz)
for i in range(batchNum):
    imgs, anss= mnist.test.next_batch(batchSz)
    testAcc += sess.run(accuracy, feed_dict={img: imgs.reshape([batchSz,28,28,1]), ans: anss})
print ("Test Accuracy: %r" % round(testAcc/batchNum, 3))

end = time.time()
print("Runtime: %g" % (end - start))

Train Accuracy: 0.989
Test Accuracy: 0.989
Runtime: 25.483
